In [ ]:
from flask import Flask, request, jsonify
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import pymongo
from bson import ObjectId  # Import ObjectId
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Fyp"]
collection = db["tourpackages"]
a_collection = db['accommodations']
t_collection = db['transports']
r_collection = db['restaurants']

# Retrieve data from MongoDB
df = pd.DataFrame(list(collection.find()))
a_df = pd.DataFrame(list(a_collection.find()))
t_df = pd.DataFrame(list(t_collection.find()))
r_df = pd.DataFrame(list(r_collection.find()))

# Feature engineering
df['total_expense_per_person'] = df['hotelRoomExpensePerPerson'] + df['transportExpensePerPerson'] + df['foodPrice']
numerical_features = ['total_expense_per_person']

a_df['hotel_expense_per_person'] = a_df['hotelRoomExpensePerPerson']
numerical_features_a = ['hotel_expense_per_person']

t_df['transport_expense_per_person'] = t_df['seatPricePerPerson']
numerical_features_t = ['transport_expense_per_person']

r_df['reservation_charges'] = r_df['reservationCharges']
numerical_features_r = ['reservation_charges']

# categorical_features = ['destination', 'province', 'country', 'transportType', 'city', 'departureCity']

# Normalize numerical features
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])
a_df[numerical_features_a] = scaler.fit_transform(a_df[numerical_features_a])
t_df[numerical_features_t] = scaler.fit_transform(t_df[numerical_features_t])
r_df[numerical_features_r] = scaler.fit_transform(r_df[numerical_features_r])

# One-hot encode categorical features
# df = pd.get_dummies(df, columns=categorical_features)

# Calculate similarity matrix
similarity_matrix = cosine_similarity(df.drop(['_id', 'name', 'contact', 'businessName', 'hotelCompanyName', 'status', 'createdAt', 
                                              'updatedAt','__v', 'userId', 'imageUrls','contact', 'summary',
                                               'inclusions', 'exclusions',
                                              'arrivalDate', 'departureDate', 'breakfast', 'lunch', 'dinner',
                                               'destination', 'province', 'country', 'transportType', 'city',
                                               'departureCity'], axis=1))
similarity_matrix_a = cosine_similarity(a_df.drop(['_id', 'name', 'userId', 'businessName', 'status', 'createdAt', 
                                                  'updatedAt', '__v', 'imageUrls', 'city', 'province', 'country',
                                                'amenities'], axis=1))
similarity_matrix_t = cosine_similarity(t_df.drop(['_id', 'userId', 'name', 'businessName', 'amenities', 'city',
                                                  'province', 'country', 'contact', 'transportType', 'capacity',
                                                  'imageUrls', 'status', 'createdAt', 'updatedAt', '__v'], axis=1))
similarity_matrix_r = cosine_similarity(r_df.drop(['_id', 'name', 'city', 'province', 'country', 'userId',
                                                  'cuisine', 'contact', 'openingTime', 'closingTime', 'reservation', 'status',
                                                  'businessName', 'imageUrls', 'createdAt', 'updatedAt', '__v'], axis=1))

# Recommendation function
def recommend_tours(price, n=10):
    # Convert 'price' column to numeric
    df['price'] = pd.to_numeric(df['price'])
    idx = df[df['price'] <= price].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    tour_indices = [i[0] for i in sim_scores]
    # Convert ObjectId to string
    return df.loc[tour_indices, '_id'].astype(str)

def recommend_accommodations(price, n=10):
    # Convert 'price' column to numeric
    a_df['hotelRoomExpensePerPerson'] = pd.to_numeric(a_df['hotelRoomExpensePerPerson'])
    idx = a_df[a_df['hotelRoomExpensePerPerson'] <= price].index[0]
    sim_scores = list(enumerate(similarity_matrix_a[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    accommodation_indices = [i[0] for i in sim_scores]
    # Convert ObjectId to string
    return a_df.loc[accommodation_indices, '_id'].astype(str)

def recommend_transports(price, n=10):
    # Convert 'price' column to numeric
    t_df['seatPricePerPerson'] = pd.to_numeric(t_df['seatPricePerPerson'])
    idx = t_df[t_df['seatPricePerPerson'] <= price].index[0]
    sim_scores = list(enumerate(similarity_matrix_t[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    transport_indices = [i[0] for i in sim_scores]
    # Convert ObjectId to string
    return t_df.loc[transport_indices, '_id'].astype(str)

def recommend_restaurants(price, n=10):
    # Convert 'price' column to numeric
    r_df['averageFoodRate'] = pd.to_numeric(r_df['averageFoodRate'])
    idx = r_df[r_df['averageFoodRate'] <= price].index[0]
    sim_scores = list(enumerate(similarity_matrix_r[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    restaurant_indices = [i[0] for i in sim_scores]
    # Convert ObjectId to string
    return r_df.loc[restaurant_indices, '_id'].astype(str)

@app.route('/recommend', methods=['POST'])
def recommend():
    price = request.json
    recommended_tours = recommend_tours(price['price'])
    return jsonify(recommended_tours.tolist())

@app.route('/recommend_accommodations', methods=['POST'])
def recommend_accommodations_endpoint():
    price = request.json
    recommended_accommodations = recommend_accommodations(price['price'])
    return jsonify(recommended_accommodations.tolist())

@app.route('/recommend_transports', methods=['POST'])
def recommend_transports_endpoint():
    price = request.json
    recommended_transports = recommend_transports(price['price'])
    return jsonify(recommended_transports.tolist())

@app.route('/recommend_restaurants', methods=['POST'])
def recommend_restaurants_endpoint():
    price = request.json
    recommended_restaurants = recommend_restaurants(price['price'])
    return jsonify(recommended_restaurants.tolist())

if __name__ == '__main__':
    app.run(port=5100)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5100/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jun/2024 03:52:26] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "OPTIONS /recommend_accommodations HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "POST /recommend_accommodations HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "OPTIONS /recommend_transports HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "POST /recommend_transports HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "OPTIONS /recommend_restaurants HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:52:26] "POST /recommend_restaurants HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:56:08] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:56:08] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:56:08] "OPTIONS /recommend_accommodations HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2024 03:56:08] "POST /recommend_accommo

In [ ]:
from flask import Flask, request, jsonify
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import pymongo
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Fyp"]
a_collection = db['accommodations']

# Retrieve data from MongoDB
a_df = pd.DataFrame(list(a_collection.find()))

# Feature engineering
numerical_features = ['hotelRoomExpensePerPerson']

# Normalize numerical features
scaler = MinMaxScaler()
a_df[numerical_features] = scaler.fit_transform(a_df[numerical_features])

# Calculate similarity matrix
similarity_matrix = cosine_similarity(a_df.drop(['_id', 'name', 'userId', 'businessName', 'status', 'createdAt', 
                                                  'updatedAt', '__v', 'imageUrls', 'city', 'province', 'country',
                                                'amenities'], axis=1))

# Recommendation function for accommodations
def recommend_accommodations(price, n=10):
    # Convert 'price' column to numeric
    a_df['hotelRoomExpensePerPerson'] = pd.to_numeric(a_df['hotelRoomExpensePerPerson'])
    idx = a_df[a_df['hotelRoomExpensePerPerson'] <= price].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    accommodation_indices = [i[0] for i in sim_scores]
    # Convert ObjectId to string
    return a_df.loc[accommodation_indices, '_id'].astype(str)

@app.route('/recommend_accommodations', methods=['POST'])
def recommend_accommodations_endpoint():
    price = request.json
    recommended_accommodations = recommend_accommodations(price['price'])
    return jsonify(recommended_accommodations.tolist())

if __name__ == '__main__':
    app.run(port=5100)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5100/ (Press CTRL+C to quit)
